# Trading Simulator in Python

## Importing Libraries

In [ ]:
from collections import deque, namedtuple
from bisect import insort
import pandas as pd
import simplefix
import socket
import time
import threading
import logging
import random
import yfinance as yf
import statistics
import numpy as np


## Implementing Order Book Functionality

In [ ]:
Order = namedtuple('Order', ['id', 'price', 'quantity', 'side', 'type', 'symbol'])

class OrderBook:
    def __init__(self):
        self.bids = {}  # Key: Price, Value: deque of Orders (bid side)
        self.asks = {}  # Key: Price, Value: deque of Orders (ask side)
        self.order_map = {}  # Key: Order ID, Value: (Order, Price level in bids/asks)

    def add_order(self, order):
        """Add a new order to the order book."""
        if order.side == 'buy':
            if order.price not in self.bids:
                self.bids[order.price] = deque()
            self.bids[order.price].append(order)
        else:
            if order.price not in self.asks:
                self.asks[order.price] = deque()
            self.asks[order.price].append(order)
        
        # Keep track of orders
        self.order_map[order.id] = order

    def remove_order(self, order_id):
        """Remove an order from the order book."""
        if order_id in self.order_map:
            order = self.order_map[order_id]
            if order.side == 'buy':
                self.bids[order.price].remove(order)
                if not self.bids[order.price]:
                    del self.bids[order.price]
            else:
                self.asks[order.price].remove(order)
                if not self.asks[order.price]:
                    del self.asks[order.price]
            del self.order_map[order_id]

    def modify_order(self, order_id, new_quantity):
        """Modify an existing order's quantity."""
        if order_id in self.order_map:
            order = self.order_map[order_id]
            self.remove_order(order_id)
            modified_order = order._replace(quantity=new_quantity)
            self.add_order(modified_order)

    def get_best_bid(self):
        """Get the highest bid price."""
        if self.bids:
            return max(self.bids)
        return None

    def get_best_ask(self):
        """Get the lowest ask price."""
        if self.asks:
            return min(self.asks)
        return None

    def cancel_order(self, order_id):
        """Cancel an existing order."""
        if order_id in self.order_map:
            self.remove_order(order_id)
            print(f"Order {order_id} has been cancelled.")
        else:
            print(f"Order {order_id} not found for cancellation.")
            
    def modify_order(self, order_id, new_quantity=None, new_price=None):
        """Modify an existing order's quantity and/or price."""
        if order_id in self.order_map:
            order = self.order_map[order_id]
            self.remove_order(order_id)
            if new_quantity is not None:
                order = order._replace(quantity=new_quantity)
            if new_price is not None:
                order = order._replace(price=new_price)
            self.add_order(order)
            print(f"Order {order_id} has been modified.")
        else:
            print(f"Order {order_id} not found for modification.")
            
    def display_bids(self):
        """Display all current bid orders in the order book."""
        print("Bid Side:")
        for price in sorted(self.bids.keys(), reverse=True):
            for order in self.bids[price]:
                print(f"Order ID: {order.id}, Price: {order.price}, Quantity: {order.quantity}")

    def display_asks(self):
        """Display all current ask orders in the order book."""
        print("Ask Side:")
        for price in sorted(self.asks.keys()):
            for order in self.asks[price]:
                print(f"Order ID: {order.id}, Price: {order.price}, Quantity: {order.quantity}")

    def display_order_book(self):
        """Display the current state of the order book (both bids and asks)."""
        print("\nOrder Book:")
        self.display_bids()
        self.display_asks()
        
    def bids_to_dataframe(self):
        """Convert the bid side of the order book to a pandas DataFrame."""
        bids = []
        for price in sorted(self.bids.keys(), reverse=True):
            for order in self.bids[price]:
                bids.append({'Order ID': order.id, 'Price': order.price, 'Quantity': order.quantity})
        return pd.DataFrame(bids)

    def asks_to_dataframe(self):
        """Convert the ask side of the order book to a pandas DataFrame."""
        asks = []
        for price in sorted(self.asks.keys()):
            for order in self.asks[price]:
                asks.append({'Order ID': order.id, 'Price': order.price, 'Quantity': order.quantity})
        return pd.DataFrame(asks)

    def display_order_book(self):
        """Display the current state of the order book as pandas DataFrames."""
        print("\nOrder Book:")
        bids_df = self.bids_to_dataframe()
        asks_df = self.asks_to_dataframe()

        print("Bid Side:")
        print(bids_df)
        print("\nAsk Side:")
        print(asks_df)

# Implementing a Matching Engine

In [ ]:
class MatchingEngine:
    def __init__(self, order_book):
        self.order_book = order_book

    def match_order(self, incoming_order):
        """Match incoming orders against the order book."""
        if incoming_order.side == 'buy':
            self.match_buy_order(incoming_order)
        else:
            self.match_sell_order(incoming_order)

    def match_buy_order(self, order):
        """Match buy orders."""
        while order.quantity > 0 and self.order_book.get_best_ask() is not None:
            best_ask_price = self.order_book.get_best_ask()
            logging.info(f"Best ask price: {best_ask_price}, Incoming buy order price: {order.price}")
            if order.price >= best_ask_price:
                logging.info(f"Executing buy order at price {best_ask_price}")
                self.execute_order(order, best_ask_price, 'sell')
            else:
                break

    def match_sell_order(self, order):
        """Match sell orders."""
        while order.quantity > 0 and self.order_book.get_best_bid() is not None:
            best_bid_price = self.order_book.get_best_bid()
            logging.info(f"Best bid price: {best_bid_price}, Incoming sell order price: {order.price}")
            if order.price <= best_bid_price:
                logging.info(f"Executing sell order at price {best_bid_price}")
                self.execute_order(order, best_bid_price, 'buy')
            else:
                break
                
    def execute_order(self, order, price, counter_side):
        """Execute orders at the given price."""
        counter_orders = self.order_book.asks if counter_side == 'sell' else self.order_book.bids
        queue = counter_orders.get(price, deque())  # Safely get the queue, return an empty deque if the price level doesn't exist
    
        while order.quantity > 0 and queue:
            best_order = queue[0]
            if best_order.quantity > order.quantity:
                # Fully match the incoming order
                best_order = best_order._replace(quantity=best_order.quantity - order.quantity)
                self.order_book.modify_order(best_order.id, best_order.quantity)
                order = order._replace(quantity=0)
                logging.info(f"Fully matched order {order.id} with {best_order.id} at price {price}")
            elif best_order.quantity <= order.quantity:
                # Partially or fully match the incoming order
                order = order._replace(quantity=order.quantity - best_order.quantity)
                self.order_book.remove_order(best_order.id)
                if queue:  # Check if queue is not empty before popping
                    queue.popleft()  # Remove the matched order from the queue
                    logging.info(f"Partially matched order {order.id} with {best_order.id} at price {price}")
            
            # Check if the queue is empty and safely remove the price level
            if not queue:
                if price in counter_orders:
                    del counter_orders[price]
                    logging.info(f"Deleted price level {price} from order book")
                else:
                    logging.warning(f"Price level {price} already removed from order book")



## Writing a Simplefix Application

In [ ]:
# Set up basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

FIX_TAGS = {
    8: "BeginString",
    35: "MsgType",
    11: "ClOrdID",
    54: "Side",
    55: "Symbol",
    44: "Price",
    38: "OrderQty",
    10: "Checksum",
    41: "OrigClOrdID"
}

SIDE_MAPPING = {
    '1': 'Buy',
    '2': 'Sell'
}

MSG_TYPE_MAPPING = {
    'D': 'NewOrderSingle',
    'F': 'OrderCancelRequest',
}

class FixApplication:
    def __init__(self, matching_engine):
        self.matching_engine = matching_engine
        self.parser = simplefix.parser.FixParser()
        self.server_socket = None

    def translate_fix_message(self, parsed_msg):
        """Translate FIX message tags to human-readable field names."""
        translated_msg = {}
        for field in parsed_msg:
            tag = field[0]
            value = field[1].decode()
            field_name = FIX_TAGS.get(tag, f"Unknown({tag})")
            if field_name == "Side":
                value = SIDE_MAPPING.get(value, value)
            if field_name == "MsgType":
                value = MSG_TYPE_MAPPING.get(value, value)
            translated_msg[field_name] = value
        return translated_msg

    def handle_new_order(self, translated_msg):
        order_id = translated_msg["ClOrdID"]
        side = translated_msg["Side"]
        price = float(translated_msg["Price"])
        quantity = int(translated_msg["OrderQty"])
        symbol = translated_msg["Symbol"]

        order = Order(id=order_id, price=price, quantity=quantity, side=side.lower(), type='limit')
        self.matching_engine.match_order(order)
        logging.info(f"New order processed: {order_id} ({side} {quantity} {symbol} @ {price})")

    def handle_cancel_order(self, translated_msg):
        order_id = translated_msg["OrigClOrdID"]
        self.matching_engine.cancel_order(order_id)
        logging.info(f"Cancel order processed for order ID: {order_id}")

    def handle_incoming_message(self, message):
        self.parser.append_buffer(message)
        parsed_msg = self.parser.get_message()

        if parsed_msg is None:
            logging.error("Received malformed message.")
            return

        # Translate the message to human-readable form
        translated_msg = self.translate_fix_message(parsed_msg)
        logging.info(f"Received FIX message: {translated_msg}")

        msg_type = translated_msg.get("MsgType")
        if msg_type == 'NewOrderSingle':
            self.handle_new_order(translated_msg)
        elif msg_type == 'OrderCancelRequest':
            self.handle_cancel_order(translated_msg)
        else:
            logging.warning(f"Unhandled message type: {msg_type}")

    def start(self):
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.server_socket.bind(('localhost', 5005))
        self.server_socket.listen(5)
        self.running = True
        logging.info("FIX server started. Waiting for connections...")

        while self.running:
            try:
                client_socket, addr = self.server_socket.accept()
                logging.info(f"Connection from {addr}")
                data = client_socket.recv(1024)
                if data:
                    logging.info(f"Received raw data: {data.decode()}")
                    self.handle_incoming_message(data)
                    client_socket.sendall(b'Ack')
                client_socket.close()
            except OSError:
                if not self.running:
                    logging.info("Server socket closed.")
                else:
                    raise

    def stop(self):
        self.running = False
        if self.server_socket:
            self.server_socket.close()
            logging.info("FIX server stopped.")

    def create_order_message(self, order):
        """Create a FIX message based on the given order."""
        msg = simplefix.FixMessage()
        msg.append_pair(8, b'FIX.4.2')
        msg.append_pair(35, b'D')  # New Order Single
        msg.append_pair(11, order['id'].encode())  # Order ID (ClOrdID)
        msg.append_pair(54, b'1' if order['side'] == 'buy' else b'2')  # Side
        msg.append_pair(55, order['symbol'].encode())  # Symbol
        msg.append_pair(44, str(order['price']).encode())  # Price
        msg.append_pair(38, str(order['quantity']).encode())  # OrderQty
        msg.append_pair(10, b'000')  # Checksum placeholder

        return msg

    def send_message(self, msg):
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect(('localhost', 5005))
        client_socket.sendall(msg.encode())
        response = client_socket.recv(1024)
        logging.info(f"Received response: {response.decode()}")
        client_socket.close()
        
    def send_execution_report(self, order, status, filled_quantity):
        """Send an execution report back to the client."""
        report = simplefix.FixMessage()
        report.append_pair(8, b'FIX.4.2')
        report.append_pair(35, b'8')  # MsgType: 8 = Execution Report
        report.append_pair(37, order.id.encode())  # OrderID
        report.append_pair(17, str(random.randint(100000, 999999)).encode())  # ExecID
        report.append_pair(150, b'0' if status == 'Filled' else b'1')  # ExecType: 0 = New, 1 = Partial fill
        report.append_pair(39, b'2' if status == 'Filled' else b'1')  # OrdStatus: 2 = Filled, 1 = Partially filled
        report.append_pair(55, order.symbol.encode())  # Symbol
        report.append_pair(54, b'1' if order.side == 'buy' else b'2')  # Side
        report.append_pair(38, str(order.quantity).encode())  # OrderQty
        report.append_pair(44, str(order.price).encode())  # Price
        report.append_pair(14, str(filled_quantity).encode())  # CumQty
        report.append_pair(6, str(order.price).encode())  # AvgPx
        report.append_pair(10, b'000')  # Checksum placeholder

        self.send_message(report)

    def handle_incoming_message(self, message):
        self.parser.append_buffer(message)
        parsed_msg = self.parser.get_message()

        if parsed_msg is None:
            logging.error("Received malformed message.")
            return

        translated_msg = self.translate_fix_message(parsed_msg)
        logging.info(f"Received FIX message: {translated_msg}")

        msg_type = translated_msg.get("MsgType")
        if msg_type == 'NewOrderSingle':
            self.handle_new_order(translated_msg)
        elif msg_type == 'OrderCancelRequest':
            self.handle_cancel_order(translated_msg)
        else:
            logging.warning(f"Unhandled message type: {msg_type}")

    def handle_new_order(self, translated_msg):
        order_id = translated_msg["ClOrdID"]
        side = translated_msg["Side"]
        price = float(translated_msg["Price"])
        quantity = int(translated_msg["OrderQty"])
        symbol = translated_msg["Symbol"]

        order = Order(id=order_id, price=price, quantity=quantity, side=side.lower(), type='limit')
        self.matching_engine.match_order(order)
        logging.info(f"New order processed: {order_id} ({side} {quantity} {symbol} @ {price})")

        # Assume that matching_engine.match_order can return the filled quantity and status
        filled_quantity = order.quantity  # Placeholder; update with actual logic
        status = 'Filled' if filled_quantity == quantity else 'PartiallyFilled'
        self.send_execution_report(order, status, filled_quantity)


## Getting Live Market Data Feed Using yfinance

In [ ]:
class MarketDataFeed:
    def __init__(self, symbol):
        self.symbol = symbol
        self.subscribers = []
        self.running = False

    def subscribe(self, client):
        """Subscribe a client to the market data feed."""
        self.subscribers.append(client)

    def broadcast(self, data):
        """Broadcast data to all subscribers."""
        for client in self.subscribers:
            client.receive(data)

    def fetch_market_data(self):
        """Fetch market data using yfinance."""
        ticker = yf.Ticker(self.symbol)
        data = ticker.history(period="1d", interval="1m")  # 1-minute interval data
        if not data.empty:
            latest_data = data.iloc[-1]
            return {
                'symbol': self.symbol,
                'timestamp': latest_data.name,
                'price': latest_data['Close'],
                'volume': latest_data['Volume']
            }
        return None

    def start(self):
        """Start the feed to fetch and broadcast market data."""
        self.running = True
        while self.running:
            market_data = self.fetch_market_data()
            if market_data:
                self.broadcast(market_data)
            time.sleep(60)  # Fetch every 1 minute

    def stop(self):
        """Stop the market data feed."""
        self.running = False

## Writing a Simple Market Maker Algorithm

In [ ]:
class MarketMaker:
    def __init__(self, symbol, matching_engine):
        self.symbol = symbol
        self.matching_engine = matching_engine
        self.spread = 0.5  # Fixed spread for simplicity
        self.price_history = []  # Track price history for volatility calculation
        self.running = False

    def calculate_dynamic_spread(self):
        """Calculate spread based on recent price volatility."""
        if len(self.price_history) < 10:
            return self.spread  # Not enough data to calculate volatility
        
        volatility = statistics.stdev(self.price_history[-10:])
        return max(self.spread, volatility * 0.5)  # Dynamic spread

    def generate_order(self, side, price):
        """Generate a market-making order."""
        order_id = str(random.randint(10000, 99999))
        quantity = random.randint(10, 100)
        order = Order(id=order_id, price=price, quantity=quantity, side=side, type='limit', symbol=self.symbol)
        return order

    def on_market_data(self, data):
        """Respond to market data by posting orders."""
        if data['symbol'] == self.symbol:
            self.price_history.append(data['price'])
            dynamic_spread = self.calculate_dynamic_spread()
            best_bid_price = data['price'] - dynamic_spread
            best_ask_price = data['price'] + dynamic_spread

            bid_order = self.generate_order('buy', best_bid_price)
            ask_order = self.generate_order('sell', best_ask_price)

            self.matching_engine.match_order(bid_order)
            self.matching_engine.match_order(ask_order)
            logging.info(f"Posted bid: {bid_order.price}, ask: {ask_order.price}")

    def start(self, feed):
        """Start the market maker."""
        self.running = True
        feed.subscribe(self)

    def stop(self):
        """Stop the market maker."""
        self.running = False

    def receive(self, data):
        """Handle incoming market data."""
        self.on_market_data(data)


## Synthetic Liquidity Generator

In [ ]:
class SyntheticLiquidityProvider:
    def __init__(self, symbol, matching_engine, num_orders):
        self.symbol = symbol
        self.matching_engine = matching_engine
        self.num_orders = num_orders

    def generate_liquidity(self):
        for _ in range(self.num_orders):
            side = 'buy' if random.random() < 0.5 else 'sell'
            ticker = yf.Ticker(self.symbol)
            data = ticker.history(period="1d", interval="1m")
            latest_data = data.iloc[-1]
            price = latest_data['Close']
            order = Order(id=str(random.randint(10000, 99999)), price=price, quantity=random.randint(10, 100), side=side, type='limit', symbol=self.symbol)
            self.matching_engine.match_order(order)
            logging.info(f"Synthetic liquidity added: {side} order at {price}")

## Backtesting

In [ ]:
def load_historical_data(symbol, start_date, end_date):
    """Load historical data from yfinance."""
    data = yf.download(symbol, start=start_date, end=end_date)
    # Ensure the data is formatted as expected
    data.reset_index(inplace=True)
    return data

def run_backtest(historical_data, market_maker, matching_engine):
    """Simulate historical data through the market maker and matching engine."""
    for index, row in historical_data.iterrows():
        # Simulate a market data update for the market maker
        market_data = {
            'symbol': market_maker.symbol,
            'price': row['Close'],
            'timestamp': row['Date']
        }
        
        # MarketMaker reacts to the incoming market data
        market_maker.on_market_data(market_data)
        
        # The MarketMaker will internally decide whether to generate an order
        # and will send it to the MatchingEngine as needed.
        
        # Optional: sleep to simulate time passing, though this isn't necessary in backtesting
        # time.sleep(1)

    print("Backtest completed.")


In [ ]:
if __name__ == "__main__":
    # Initialize historical data
    symbol = "AAPL"
    start_date = "2023-01-01"
    end_date = "2023-12-31"
    historical_data = load_historical_data(symbol, start_date, end_date)

    # Initialize the market maker, matching engine, and other components
    order_book = OrderBook()
    matching_engine = MatchingEngine(order_book)
    market_maker = MarketMaker(symbol=symbol, matching_engine=matching_engine)

    # Run the backtest
    run_backtest(historical_data, market_maker, matching_engine)

## How to Run/Test

### Initialising Everything

In [ ]:
order_book = OrderBook()
matching_engine = MatchingEngine(order_book)
fix_app = FixApplication(matching_engine)

# Prompt user for the stock ticker
ticker_symbol = input("Enter the stock ticker symbol you want to track (e.g., AAPL): ").upper()

# Initialize the MarketDataFeed with the user-provided ticker symbol
market_data_feed = MarketDataFeed(symbol=ticker_symbol)
market_maker = MarketMaker(symbol=ticker_symbol, matching_engine=matching_engine)

# Start FIX server
fix_thread = threading.Thread(target=fix_app.start)
fix_thread.start()

# Start market data feed
feed_thread = threading.Thread(target=market_data_feed.start)
feed_thread.start()

# Start market maker
market_maker.start(market_data_feed)

### Order Creation and Control

In [ ]:
# Create an ask order
ask_order = Order(id='2', price=415, quantity=100, side='sell', type='limit', symbol='MSFT')
order_book.add_order(ask_order)

#Post a buy order 
buy_order = Order(id='2', price=228.00, quantity=100, side='buy', type='limit', symbol='AAPL')
order_book.add_order(buy_order)

#matching_engine.match_order(buy_order)

# Display the order book at any time
order_book.display_order_book()

# Market data feed updates
market_data_feed.subscribe(your_market_data_receiver_function)


In [ ]:
order1 = Order(id='1', price=100.0, quantity=10, side='buy', type='limit', symbol='AAPL')
order2 = Order(id='2', price=101.0, quantity=20, side='buy', type='limit', symbol='AAPL')
order3 = Order(id='3', price=102.0, quantity=15, side='sell', type='limit', symbol='AAPL')
order4 = Order(id='4', price=103.0, quantity=25, side='sell', type='limit', symbol='AAPL')

In [ ]:
order_book.modify_order(order_id='2', new_quantity=30, new_price=100.5)
order_book.cancel_order(order_id='3')


### Synthetic Liquidity Controls

In [ ]:
liquidity_provider = SyntheticLiquidityProvider(symbol=symbol, matching_engine=matching_engine, num_orders=10)
liquidity_provider.generate_liquidity()


In [ ]:
liquidity_thread = threading.Thread(target=auto_inject_liquidity, args=(liquidity_provider, 5))  # Inject every 5 seconds
liquidity_thread.start()

### Stop All Processes

In [ ]:
market_maker.stop()
market_data_feed.stop()
fix_app.stop()